In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
owid_df = pd.read_csv('../data/energy-data/owid-energy-data.csv')
gov_response = pd.read_csv('../data/covid-policy-dataset/data/timeseries_indices/OxCGRT_timeseries_GovernmentResponseIndex_v1.csv')

In [3]:
cov_compact = pd.read_csv('../data/covid-policy-dataset/data/OxCGRT_compact_national_v1.csv')

In [4]:
gov_response_countries = gov_response[gov_response['Jurisdiction']=='NAT_TOTAL']

In [5]:
gov_response_rotate = gov_response_countries[gov_response_countries['CountryName'].isin(owid_df['country'])].drop(columns=['CountryCode'
                                                                                                          ,'RegionCode'
                                                                                                          ,'RegionName'
                                                                                                          ,'CityCode'
                                                                                                          ,'CityName'
                                                                                                          ,'Jurisdiction']).transpose().reset_index()

In [6]:
gov_response_rotate.columns = gov_response_rotate.iloc[0]

In [7]:
gov_response_rotate = gov_response_rotate.drop(index=0).rename(columns={'CountryName':'Date'}).reset_index(drop=True)

In [8]:
gov_num = gov_response_rotate.iloc[:,1:].apply(pd.to_numeric)

In [9]:
gov_date = gov_response_rotate.iloc[:,0].apply(pd.to_datetime)

In [10]:
gov_num.insert(0,'Date',gov_date)

In [11]:
gov_num['Year'] = gov_num['Date'].dt.year

In [12]:
gov_mean = gov_num.groupby('Year').mean().reset_index().drop(columns='Date')

In [13]:
gov_melt = gov_mean.melt(id_vars=['Year'],value_vars=gov_mean.columns[1:]).rename(columns={0:'Country'})

In [14]:
owid_trim = owid_df[['country',
                     'year',
                     'coal_consumption',
                     'hydro_consumption',
                     'oil_consumption',
                     'nuclear_consumption',
                     'gas_consumption']]

In [23]:
cov_compact['Year'] = cov_compact['Date'].astype(str).str[:4]

In [24]:
cov_compact['Year'] = cov_compact['Year'].astype(int)

In [46]:
cov_compact.drop(columns=['CountryCode','RegionName','RegionCode','Jurisdiction','Date'])

,CountryName,C1M_School closing,C1M_Flag,C2M_Workplace closing,C2M_Flag,C3M_Cancel public events,C3M_Flag,C4M_Restrictions on gatherings,C4M_Flag,C5M_Close public transport,...,V4_Mandatory Vaccination (summary),ConfirmedCases,ConfirmedDeaths,MajorityVaccinated,PopulationVaccinated,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex,Year
0,Aruba,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0,2020
1,Aruba,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0,2020
2,Aruba,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0,2020
3,Aruba,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0,2020
4,Aruba,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202755,Zimbabwe,1.0,1.0,2.0,1.0,1.0,1.0,0.0,NaN,0.0,...,1.0,259981.0,5637.0,NV,29.11,29.48,36.44,41.65,0.0,2022
202756,Zimbabwe,1.0,1.0,2.0,1.0,1.0,1.0,0.0,NaN,0.0,...,1.0,259981.0,5637.0,NV,29.11,29.48,36.44,41.65,0.0,2022
202757,Zimbabwe,1.0,1.0,2.0,1.0,1.0,1.0,0.0,NaN,0.0,...,1.0,259981.0,5637.0,NV,29.11,29.48,36.44,41.65,0.0,2022
202758,Zimbabwe,1.0,1.0,2.0,1.0,1.0,1.0,0.0,NaN,0.0,...,1.0,259981.0,5637.0,NV,29.11,29.48,36.44,41.65,0.0,2022


In [47]:
cov_compact_test = cov_compact[cov_compact.columns.drop(list(cov_compact.filter(regex='Flag')))]

In [48]:
cov_compact = cov_compact_test.drop(columns=['CountryCode',
                                             'RegionName',
                                             'RegionCode',
                                             'Jurisdiction',
                                             'Date',
                                             'V2B_Vaccine age eligibility/availability age floor (general population summary)',
                                            'V2C_Vaccine age eligibility/availability age floor (at risk summary)',
                                            'MajorityVaccinated'])

In [49]:
cov_mean = cov_compact.groupby(['CountryName','Year']).mean().reset_index()

In [50]:
owid_trim = owid_trim[(owid_trim['country'].isin(cov_mean['CountryName']))&
                    (owid_trim['year'].between(2019,2023))].rename(columns={'country':'CountryName',
                                                                            'year':'Year'})

In [51]:
owid_trim = owid_trim[owid_trim['gas_consumption'].notnull()]

In [52]:
cov_owid_merge = owid_trim.merge(cov_mean,how='outer',on=('CountryName','Year'))

In [53]:
cov_owid_merge = cov_owid_merge[cov_owid_merge['gas_consumption'].notnull()]

In [54]:
#cov_owid_merge.to_csv('cov_owid_merge.csv',index=False)

In [62]:
#cov_compact[cov_compact['CountryName'].isin(gov_owid_merge['Country'])]

In [288]:
owid_per_capita = (
owid_df[['country',
        'year',
        'coal_cons_per_capita',
        'hydro_energy_per_capita',
        'oil_energy_per_capita',
        'nuclear_energy_per_capita',
        'gas_energy_per_capita']])

In [298]:
owid_per_capita = owid_per_capita[owid_per_capita['country'].isin(change_calc['CountryName'])]

In [64]:
owid_sub = owid_trim[(owid_trim['Year'].isin([2019,2021]))&(owid_trim['gas_consumption'].notnull())]

In [78]:
owid_trim_countries = owid_sub[['CountryName','Year']].drop(columns='Year')

In [80]:
owid_sub_2 = owid_sub.groupby('CountryName').diff().dropna().drop(columns='Year').reset_index()

In [178]:
change_calc = owid_trim_countries.reset_index().merge(owid_sub_2,how='inner',on='index').drop(columns='index')

In [180]:
change_calc[change_calc.columns[2:]] = change_calc.iloc[:,2:]/2

In [182]:
change_calc.columns = change_calc.columns.str.replace('_consumption','')

In [206]:
change_calc.insert(1,'flag','calc')

In [208]:
change_calc

,CountryName,flag,coal,hydro,oil,nuclear,gas
0,Algeria,calc,-1.698,-0.1885,-7.8735,0.0000,15.4915
1,Argentina,calc,5.928,-10.5445,14.5295,2.0910,-3.1240
2,Australia,calc,-33.945,2.3795,-34.1420,0.0000,-22.4065
3,Austria,calc,-3.783,-3.1080,-6.4930,0.0000,0.5750
4,Azerbaijan,calc,-0.006,-0.3915,1.0900,0.0000,5.1410
...,...,...,...,...,...,...,...
60,United Kingdom,calc,-4.671,-0.7565,-86.0685,-13.4265,-4.7355
61,United States,calc,-213.566,-50.5150,-209.3490,-47.2235,-72.8750
62,Uzbekistan,calc,4.898,-1.9620,1.6725,0.0000,9.6385
63,Venezuela,calc,-0.323,6.0090,-44.6490,0.0000,12.4145


In [90]:
cov_mean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (total 37 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   CountryName                                         555 non-null    object 
 1   Year                                                555 non-null    int32  
 2   C1M_School closing                                  555 non-null    float64
 3   C2M_Workplace closing                               555 non-null    float64
 4   C3M_Cancel public events                            555 non-null    float64
 5   C4M_Restrictions on gatherings                      555 non-null    float64
 6   C5M_Close public transport                          555 non-null    float64
 7   C6M_Stay at home requirements                       555 non-null    float64
 8   C7M_Restrictions on internal movement               555 non-null    float64
 9  

In [ ]:
#cov_mean.to_csv('cov_mean.csv',index=False)

In [92]:
owid_cut = owid_df[['country',
                     'year',
                     'coal_consumption',
                     'hydro_consumption',
                     'oil_consumption',
                     'nuclear_consumption',
                     'gas_consumption']]

In [ ]:
#owid_cut[(owid_cut['country'].isin(cov_mean['CountryName']))&(owid_cut['gas_consumption'].notnull())].to_csv('owid_cut.csv',index=False)

In [94]:
owid_cut

,country,year,coal_consumption,hydro_consumption,oil_consumption,nuclear_consumption,gas_consumption
0,ASEAN (Ember),2000,NaN,NaN,NaN,NaN,NaN
1,ASEAN (Ember),2001,NaN,NaN,NaN,NaN,NaN
2,ASEAN (Ember),2002,NaN,NaN,NaN,NaN,NaN
3,ASEAN (Ember),2003,NaN,NaN,NaN,NaN,NaN
4,ASEAN (Ember),2004,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
21807,Zimbabwe,2019,NaN,NaN,NaN,0.0,NaN
21808,Zimbabwe,2020,NaN,NaN,NaN,0.0,NaN
21809,Zimbabwe,2021,NaN,NaN,NaN,0.0,NaN
21810,Zimbabwe,2022,NaN,NaN,NaN,0.0,NaN


In [186]:
fuel_change = owid_df['country'].reset_index().merge(owid_cut[(owid_cut['country'].isin(cov_mean['CountryName']))&
                    (owid_cut['gas_consumption'].notnull())&
                    (owid_cut['year'].isin([2019,2020]))].groupby(['country'])[['hydro_consumption',
                                                                                'coal_consumption',
                                                                                'oil_consumption',
                                                                                'nuclear_consumption',
                                                                                'gas_consumption']].diff()
                                                     .dropna()
                                                     .reset_index(),how='inner',on='index').drop(columns='index')

In [190]:
fuel_change.columns = fuel_change.columns.str.replace('_consumption','')

In [194]:
fuel_change.nsmallest(1,'nuclear')

,country,hydro,coal,oil,nuclear,gas
20,France,14.364,-23.441,-144.009,-117.566,-30.792


In [132]:
owid_cut[(owid_cut['country'].isin(cov_mean['CountryName']))&
                    (owid_cut['gas_consumption'].notnull())&
                    (owid_cut['year'].isin([2019,2020]))&
                    (owid_cut['country']=='China')]

,country,year,coal_consumption,hydro_consumption,oil_consumption,nuclear_consumption,gas_consumption
4501,China,2019,22922.236,3354.375,7891.510,882.026,3083.768
4502,China,2020,23402.553,3471.190,7959.667,922.889,3366.173


In [204]:
fuel_change.merge(change_calc,how='inner',left_on='country',right_on='CountryName',suffixes=('_actual','_calc')).drop(columns='CountryName')

,country,hydro_actual,coal_actual,oil_actual,nuclear_actual,gas_actual,coal_calc,hydro_calc,oil_calc,nuclear_calc,gas_calc
0,Algeria,-0.271,-3.279,-25.026,0.000,-17.046,-1.698,-0.1885,-7.8735,0.0000,15.4915
1,Argentina,-10.338,1.891,-33.637,3.875,-26.277,5.928,-10.5445,14.5295,2.0910,-3.1240
2,Australia,0.916,-18.331,-81.850,0.000,-17.397,-33.945,2.3795,-34.1420,0.0000,-22.4065
3,Austria,2.683,-4.913,-20.172,0.000,-3.732,-3.783,-3.1080,-6.4930,0.0000,0.5750
4,Azerbaijan,-1.316,-0.003,-5.408,0.000,5.776,-0.006,-0.3915,1.0900,0.0000,5.1410
...,...,...,...,...,...,...,...,...,...,...,...
60,United Kingdom,2.400,-6.127,-196.564,-15.406,-47.284,-4.671,-0.7565,-86.0685,-13.4265,-4.7355
61,United States,-9.822,-594.279,-1252.363,-59.728,-164.503,-213.566,-50.5150,-209.3490,-47.2235,-72.8750
62,Uzbekistan,-3.905,7.746,4.213,0.000,-10.144,4.898,-1.9620,1.6725,0.0000,9.6385
63,Venezuela,10.949,-0.262,-99.539,0.000,-40.151,-0.323,6.0090,-44.6490,0.0000,12.4145


In [210]:
fuel_change.insert(1,'flag','actual')

In [220]:
pd.concat([(fuel_change),(change_calc.rename(columns={'CountryName':'country'}))]).sort_values(by='country').to_csv('fuel_change.csv',index=False)

In [272]:
fuel_diff = ((fuel_change.iloc[:,2:])-(change_calc.iloc[:,2:]))

In [274]:
fuel_diff.insert(0,'country',fuel_change['country'])

In [276]:
fuel_diff.insert(1,'flag','diff')

In [278]:
fuel_diff.to_csv('fuel_diff.csv',index=False)

In [326]:
fuel_change_per_capita = owid_df['country'].reset_index().merge(owid_per_capita[(owid_per_capita['country'].isin(cov_mean['CountryName']))&
                    (owid_per_capita['gas_energy_per_capita'].notnull())&
                    (owid_per_capita['year'].isin([2019,2020]))].groupby(['country'])[['hydro_energy_per_capita',
                                                                                'coal_cons_per_capita',
                                                                                'oil_energy_per_capita',
                                                                                'nuclear_energy_per_capita',
                                                                                'gas_energy_per_capita']].diff()
                                                     .dropna()
                                                     .reset_index(),how='inner',on='index').drop(columns='index')

In [330]:
fuel_change_per_capita.insert(1,'flag','actual')

In [336]:
fuel_change_per_capita_calc = owid_df['country'].reset_index().merge(
    owid_per_capita[(owid_per_capita['year'].isin([2019,2021]))&(owid_per_capita['gas_energy_per_capita'].notnull())]
        .groupby('country')[[
                 'coal_cons_per_capita',
              'hydro_energy_per_capita',
                'oil_energy_per_capita',
            'nuclear_energy_per_capita',
                'gas_energy_per_capita']].diff().dropna().reset_index(),how='inner',on='index').drop(columns='index')

In [338]:
fuel_change_per_capita_calc.insert(1,'flag','expected')

In [340]:
fuel_change_per_capita.iloc[:,2:]

,hydro_energy_per_capita,coal_cons_per_capita,oil_energy_per_capita,nuclear_energy_per_capita,gas_energy_per_capita
0,-6.384,-77.410,-671.245,0.000,-573.496
1,-239.994,40.826,-793.296,82.973,-650.582
2,17.885,-947.892,-3478.752,0.000,-889.488
3,263.302,-563.468,-2318.423,0.000,-450.278
4,-129.994,-0.304,-557.094,0.000,503.086
...,...,...,...,...,...
60,34.812,-95.685,-2984.718,-238.640,-753.933
61,-40.105,-1814.499,-3876.515,-208.921,-612.604
62,-124.929,220.648,96.765,0.000,-524.305
63,480.809,-8.675,-3356.955,0.000,-1260.177


In [342]:
per_capita_diff = ((fuel_change_per_capita.iloc[:,2:])-(fuel_change_per_capita_calc.iloc[:,2:]))

In [346]:
per_capita_diff.insert(0,'country',fuel_change_per_capita['country'])

In [348]:
per_capita_diff.insert(1,'flag','diff')

In [350]:
per_capita_diff

,country,flag,coal_cons_per_capita,gas_energy_per_capita,hydro_energy_per_capita,nuclear_energy_per_capita,oil_energy_per_capita
0,Algeria,diff,-35.226,-923.148,2.447,0.000,-129.836
1,Argentina,diff,-87.982,-390.482,244.790,-3.789,-1350.694
2,Australia,diff,778.944,1217.339,-133.979,0.000,-326.426
3,Austria,diff,-121.423,-531.822,1017.297,0.000,-781.404
4,Azerbaijan,diff,0.297,-404.283,-50.985,0.000,-720.508
...,...,...,...,...,...,...,...
60,United Kingdom,diff,-18.564,-526.096,59.054,176.370,-330.785
61,United States,diff,-1105.885,22.168,277.574,122.565,-2389.552
62,Uzbekistan,diff,97.480,-651.631,6.944,0.000,55.738
63,Venezuela,diff,1.949,-2382.259,-101.732,0.000,-411.998


In [356]:
pd.concat([fuel_change_per_capita,fuel_change_per_capita_calc,per_capita_diff]).sort_values(by='country').to_csv('change_per_capita.csv',index=False)